## GPT Index <> Langchain Integrations

This demo notebook shows how you can provide integrations between GPT Index and Langchain. It provides the following examples:
- Using GPT Index as a callable tool with a Langchain agent
- Using GPT Index as a memory module; this allows you to insert arbitrary amounts of conversation history with a Langchain chatbot!

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Using GPT Index as a Callable Tool

In [2]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

In [3]:
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()
index = GPTVectorStoreIndex.from_documents(documents=documents)

In [4]:
tools = [
    Tool(
        name = "GPT Index",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about the author. The input to this tool should be a complete english sentence.",
        return_direct=True
    ),
]

In [5]:
# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0)
agent_executor = initialize_agent(tools, llm, agent="conversational-react-description", memory=memory)

In [6]:
agent_executor.run(input="hi, i am bob")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Hi Bob, nice to meet you! How can I help you today?

> Finished chain.


'Hi Bob, nice to meet you! How can I help you today?'

In [7]:
agent_executor.run(input="What did the author do growing up?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: GPT Index
Action Input: What did the author do growing up?> [query] Total LLM token usage: 3841 tokens
> [query] Total embedding token usage: 8 tokens

Observation: 

The author grew up writing short stories, programming on an IBM 1401, and building a computer kit with a friend. He also wrote simple games, a program to predict how high his model rockets would fly, and a word processor. He studied philosophy in college, but switched to AI and taught himself Lisp. He wrote a book about Lisp hacking and reverse-engineered SHRDLU. He also took art classes at Harvard and applied to art schools, but was disappointed by the lack of teaching and learning in the painting department at the Accademia. He also had experience with 19th century studio painting conventions, such as having a little stove fed with kindling and a nude model sitting as close to it as possible.


> Finished chain.


'\n\nThe author grew up writing short stories, programming on an IBM 1401, and building a computer kit with a friend. He also wrote simple games, a program to predict how high his model rockets would fly, and a word processor. He studied philosophy in college, but switched to AI and taught himself Lisp. He wrote a book about Lisp hacking and reverse-engineered SHRDLU. He also took art classes at Harvard and applied to art schools, but was disappointed by the lack of teaching and learning in the painting department at the Accademia. He also had experience with 19th century studio painting conventions, such as having a little stove fed with kindling and a nude model sitting as close to it as possible.'

#### Using GPT Index as a memory module

In [12]:
# try using GPT List Index!
from langchain import OpenAI
from langchain.llms import OpenAIChat
from langchain.agents import initialize_agent

from llama_index import GPTListIndex
from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory

In [13]:
index = GPTListIndex([])

> [build_index_from_documents] Total LLM token usage: 0 tokens
> [build_index_from_documents] Total embedding token usage: 0 tokens


In [14]:
# set Logging to DEBUG for more detailed outputs
# NOTE: you can also use a conversational chain

memory = GPTIndexChatMemory(
    index=index, 
    memory_key="chat_history", 
    query_kwargs={"response_mode": "compact"},
    # return_source returns source nodes instead of querying index
    return_source=True,
    # return_messages returns context in message format
    return_messages=True
)
llm = OpenAIChat(temperature=0)
# llm=OpenAI(temperature=0)
agent_executor = initialize_agent([], llm, agent="conversational-react-description", memory=memory)

In [15]:
agent_executor.run(input="hi, i am bob")

> [query] Total LLM token usage: 0 tokens
> [query] Total embedding token usage: 0 tokens


> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Hi Bob, nice to meet you! How can I help you today?

> Finished chain.
> Adding chunk: Human: hi, i am bob
AI: Hi Bob, nice to meet yo...
> [insert] Total LLM token usage: 0 tokens
> [insert] Total embedding token usage: 0 tokens


'Hi Bob, nice to meet you! How can I help you today?'

In [16]:
# NOTE: the query now calls the GPTListIndex memory module. 
agent_executor.run(input="what's my name?")

> [query] Total LLM token usage: 64 tokens
> [query] Total embedding token usage: 0 tokens


> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Your name is Bob.

> Finished chain.
> Adding chunk: Human: what's my name?
AI: Your name is Bob....
> [insert] Total LLM token usage: 0 tokens
> [insert] Total embedding token usage: 0 tokens


'Your name is Bob.'